## STEP 1: Import and Install necessary files

In [32]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [34]:
pip install llm_connect


In [35]:
pip show torch

Name: torch
Version: 2.1.2
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-nccl-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, bitsandbytes, fastai, llm_connect, torchaudio, torchvision


In [36]:
pip install torch==2.1.2


In [37]:
pip install faker

In [38]:
pip install fpdf

In [39]:
pip install rapidfuzz

In [40]:
!pip install openpyxl

In [42]:
pip install ezdxf

## Gen AI-Powered Procurement Automation:

We aim to streamline the end-to-end procurement lifecycle using Gen AI — from scope identification to supplier performance review. By integrating CAD parsing, historical data mining, NLP-based document generation, and AI-assisted vendor engagement, this initiative seeks to automate repetitive workflows, improve decision quality, and reduce cycle time. The final goal is to create an intelligent procurement assistant that can interpret engineering inputs, prepare RFQs, evaluate bids, and even track shipments autonomously.

## Part 1: Creating a Scope Document from `.dxf` and Current BOM Files

### Scenario Overview:

We are a part of the **procurement automation team** at a manufacturing company working on a live sourcing requirement. The **design engineering team** has shared actual `.dxf` CAD drawings for a new industrial assembly, and we also have access to the **current BOM files** from the design management system.

Our first task is to **extract key components, specifications, and material details** from these files to generate a clean **Scope Document**. This document becomes the backbone for the next stage: **RFQ (Request for Quotation) Preparation** and **intelligent vendor communication**.


Our Goal for now is to simulate data for PART 1 AND PART 2

[Engineering BOM + dxf] + [Historical BOM]

            ⬇️
     ➤ Scope Document (.xlsx)

            ⬇️

[+ RFQ Metadata + Vendor List]

            ⬇️

     ➤ RFQ Documents + Emails + Tracker


### TASK 1: Simulate BOM Files - (Using Faker Library):

To simulate realistic procurement scenarios, we generated multiple Engineering BOMs (Bill of Materials) using Python, Faker, and FPDF. Each BOM contains synthetic data like item descriptions, quantities, specs, and drawing references, saved in both Excel and PDF formats. This setup helps mimic live sourcing conditions and supports downstream tasks like RFQ preparation, AI-based parsing, and vendor communication.

In [41]:
import random
from faker import Faker
from fpdf import FPDF
import pandas as pd
import os

# Initialize Faker
tool = Faker()

# Sample company names and project names
supplier_names = ["Zenith Engineering", "Pinnacle Systems", "Vertex Constructors", "Nexus EPC"]
project_names = ["Project Apollo", "Sunrise Refinery", "Delta Wind Farm", "Oceanic Bridge", "Metro Line 9"]

# U.S.-specific units of measure and material categories
units_of_measure = ["Each", "Feet", "Pound", "Gallon"]
material_categories = ["Piping", "Electrical", "Civil", "Mechanical", "Structural"]

# Directory to save files
output_dir = "generated_files"
os.makedirs(output_dir, exist_ok=True)

# Generate a single fake Engineering BOM and save as Excel
def generate_fake_bom_excel(company, project, filename, is_bechtel=True, num_items=10):
    # Generate fake BOM data
    data = []
    for _ in range(num_items):
        item_id = f"ITM-{random.randint(1000,9999)}"
        desc = random.choice([
            "Steel Beam", "Electric Motor", "PVC Pipe", "Hydraulic Pump",
            "Control Valve", "Sensor", "Cable Reel", "Safety Helmet",
            "Heat Exchanger", "Solar Panel"
        ])
        qty = random.randint(1, 500)
        uom = random.choice(units_of_measure)
        spec = random.choice([
            "A36 Steel", "IP65 Rated", "Schedule 40", "316 Stainless Steel",
            "VFD Compatible", "High Voltage 480V"
        ])
        material_cat = random.choice(material_categories)
        drawing_ref = f"DRW-{random.randint(100,999)}-{random.choice(['PIP','ELC','CIV','MEC','STR'])}-{random.randint(1000,9999)}"
        delivery = tool.date_between(start_date='+10d', end_date='+60d').strftime('%Y-%m-%d')
        data.append([
            item_id, desc, qty, uom, spec, material_cat, drawing_ref, delivery
        ])

    # Create DataFrame
    columns = ["Item ID", "Description", "Quantity", "Unit of Measure", "Specification", "Material Category", "Drawing Reference", "Required Delivery Date"]
    df = pd.DataFrame(data, columns=columns)

    # Save as Excel
    excel_path = os.path.join(output_dir, filename + ".xlsx")
    df.to_excel(excel_path, index=False)
    print(f"Excel file saved: {excel_path}")

    # Also save as PDF
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    # Header
    pdf.cell(0, 10, f"{company} - {project}", ln=True, align='C')
    pdf.cell(0, 10, "Engineering Bill of Materials (EBOM)", ln=True, align='C')
    pdf.ln(10)

    # Table headers
    pdf.set_font("Arial", 'B', 9)
    col_widths = [20, 40, 15, 20, 40, 30, 40, 30]
    headers = columns
    for i, header in enumerate(headers):
        pdf.cell(col_widths[i], 10, header, border=1)
    pdf.ln()

    # Table rows with slight random font variation
    for _, row in df.iterrows():
        pdf.set_font("Arial", '', random.choice([8,9,10]))
        for i, item in enumerate(row):
            pdf.cell(col_widths[i], 8, str(item), border=1)
        pdf.ln()

    pdf_path = os.path.join(output_dir, filename + ".pdf")
    pdf.output(pdf_path)
    print(f"PDF file saved: {pdf_path}")

# Generate multiple Excel and PDF BOMs
def generate_multiple_boms(num_boms=5):
    for i in range(num_boms):
        if random.random() < 0.7:
            company = "Bechtel Solutions"
            is_bechtel = True
        else:
            company = random.choice(supplier_names)
            is_bechtel = False

        project = random.choice(project_names)
        filename = f"BOM_{i+1}"
        generate_fake_bom_excel(company, project, filename, is_bechtel=is_bechtel)

if __name__ == "__main__":
    generate_multiple_boms(num_boms=5)
    print(f"Generated Excel and PDF BOMs saved in {output_dir}/")


Excel file saved: generated_files/BOM_1.xlsx
PDF file saved: generated_files/BOM_1.pdf
Excel file saved: generated_files/BOM_2.xlsx
PDF file saved: generated_files/BOM_2.pdf
Excel file saved: generated_files/BOM_3.xlsx
PDF file saved: generated_files/BOM_3.pdf
Excel file saved: generated_files/BOM_4.xlsx
PDF file saved: generated_files/BOM_4.pdf
Excel file saved: generated_files/BOM_5.xlsx
PDF file saved: generated_files/BOM_5.pdf
Generated Excel and PDF BOMs saved in generated_files/


### Task 2: Generating `.dxf` Files from BOM Data

In this task, we reverse-engineer `.dxf` CAD drawings from Excel-based BOM files to simulate visual part representations. For each item in the BOM, we create a simple `.dxf` file containing geometric shapes (rectangles) and annotated metadata (Item ID, Description, Spec, Quantity). This is achieved using the `ezdxf` Python library, and helps us build a testable dataset for downstream CAD parsing or Gen AI workflows.

In [43]:
import pandas as pd
import ezdxf
import os

def generate_dxf_from_bom_folder(bom_folder="generated_files", output_dir="generated_dxf"):
    """
    Converts all Excel BOM files in the given folder to .dxf files
    using relative paths. Each item in each BOM becomes a labeled drawing.

    Parameters:
    - bom_folder: relative path to the folder containing .xlsx BOM files
    - output_dir: relative path to the output folder for .dxf files
    """
    # Get absolute paths from relative
    bom_folder = os.path.abspath(bom_folder)
    output_dir = os.path.abspath(output_dir)
    os.makedirs(output_dir, exist_ok=True)

    for bom_file in os.listdir(bom_folder):
        if bom_file.endswith(".xlsx"):
            bom_path = os.path.join(bom_folder, bom_file)
            df = pd.read_excel(bom_path)

            for index, row in df.iterrows():
                item_id = row.get("Item ID", f"ITEM_{index}")
                desc = row.get("Description", "N/A")
                spec = row.get("Specification", "N/A")
                qty = row.get("Quantity", "N/A")

                doc = ezdxf.new(dxfversion='R2010')
                msp = doc.modelspace()

                # Drawing: rectangle + annotated text
                msp.add_lwpolyline([(0, 0), (0, 10), (20, 10), (20, 0), (0, 0)], close=True)
                msp.add_text(f"{item_id}", dxfattribs={"height": 2}).dxf.insert = (1, 11)
                msp.add_text(f"{desc}", dxfattribs={"height": 1}).dxf.insert = (1, 9)
                msp.add_text(f"Spec: {spec}", dxfattribs={"height": 1}).dxf.insert = (1, 7)
                msp.add_text(f"Qty: {qty}", dxfattribs={"height": 1}).dxf.insert = (1, 5)

                base_name = os.path.splitext(bom_file)[0]
                file_name = f"{base_name}_{item_id}.dxf"
                dxf_path = os.path.join(output_dir, file_name)
                doc.saveas(dxf_path)

    print(f"All .dxf files saved to: {os.path.relpath(output_dir)}")

# Example usage (runs relative to your current working directory):
generate_dxf_from_bom_folder()


All .dxf files saved to: generated_dxf


### Task 3: Create Historical BOM Data:

In this task, we simulate historical BOM files using Python, `Faker`, and `FPDF` to mimic past project data for reference and matching. These historical records are stored in the `historical_data` folder and contain realistic combinations of descriptions, specs, categories, and delivery timelines. This data serves as the foundation for recommending specs or preferred suppliers during Scope Document creation and RFQ automation.


In [16]:
import pandas as pd
import random
import os
from faker import Faker
from fpdf import FPDF

# Initialize Faker
faker = Faker()

# Output folder
output_folder = "historical_data"
os.makedirs(output_folder, exist_ok=True)

# Sample values
descriptions = [
    "Steel Beam", "Electric Motor", "PVC Pipe", "Hydraulic Pump",
    "Control Valve", "Sensor", "Cable Reel", "Safety Helmet",
    "Heat Exchanger", "Solar Panel"
]
specs = [
    "A36 Steel", "IP65 Rated", "Schedule 40", "316 Stainless Steel",
    "VFD Compatible", "High Voltage 480V"
]
units = ["Each", "Feet", "Pound", "Gallon"]
categories = ["Piping", "Electrical", "Civil", "Mechanical", "Structural"]
suppliers = ["FlowMax Inc.", "Valvex Corp.", "IronWorks Ltd.", "GreenVolt Systems", "HydroMech Supplies"]

# Generate both Excel and PDF
def generate_historical_bom(file_name_base, num_items=10):
    data = []
    for _ in range(num_items):
        item_id = f"ITM-{random.randint(1000, 9999)}"
        desc = random.choice(descriptions)
        spec = random.choice(specs)
        qty = random.randint(10, 500)
        unit = random.choice(units)
        cat = random.choice(categories)
        drawing = f"DRW-{random.randint(100, 999)}-{random.choice(['PIP','ELC','CIV','MEC','STR'])}-{random.randint(1000, 9999)}"
        delivery = faker.date_between(start_date='-2y', end_date='today').strftime('%Y-%m-%d')
        supplier = random.choice(suppliers)
        data.append([item_id, desc, qty, unit, spec, cat, drawing, delivery, supplier])

    columns = [
        "Item ID", "Description", "Quantity", "Unit of Measure", "Specification",
        "Material Category", "Drawing Reference", "Required Delivery Date", "Supplier"
    ]
    df = pd.DataFrame(data, columns=columns)

    # Save Excel
    excel_path = os.path.join(output_folder, f"{file_name_base}.xlsx")
    df.to_excel(excel_path, index=False)

    # Generate PDF
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=10)
    pdf.cell(0, 10, f"Engineering BOM - {file_name_base}", ln=True, align='C')
    pdf.ln(5)

    # Table headers
    pdf.set_font("Arial", 'B', 8)
    col_widths = [20, 30, 12, 18, 35, 20, 35, 25, 35]
    for i, header in enumerate(columns):
        pdf.cell(col_widths[i], 6, header, border=1)
    pdf.ln()

    # Table rows
    pdf.set_font("Arial", '', 7)
    for _, row in df.iterrows():
        for i, item in enumerate(row):
            pdf.cell(col_widths[i], 6, str(item), border=1)
        pdf.ln()

    pdf_path = os.path.join(output_folder, f"{file_name_base}.pdf")
    pdf.output(pdf_path)

# Generate 10 historical BOMs
for i in range(1, 11):
    file_base = f"Old_BOM_{i}"
    generate_historical_bom(file_base)

print("✅ 10 simulated historical BOMs created in both Excel and PDF formats in 'historical_data/'")


✅ 10 simulated historical BOMs created in both Excel and PDF formats in 'historical_data/'


## ✅ Final Output: Scope Document (Enriched with Recommendations)

Based on the simulated BOMs, `.dxf` visuals, and historical BOM data using the functional notebook, we generate a **Scope Document** that is cleaned, structured, and enriched with procurement intelligence.

This document consolidates:
- BOM item details
- CAD-derived specifications
- Historical supplier matches
- Past pricing benchmarks
- Delivery timelines
- Category and risk tagging

It serves as the **final output of Scope Identification** and feeds directly into **RFQ Preparation**.

---

### 🔍 Sample Output Table

| Item Code | Item Description | Specification     | Qty | UOM  | Delivery Date | Suggested Supplier   | Past Unit Cost | Category    | Risk Flag |
|-----------|------------------|--------------------|-----|------|----------------|-----------------------|----------------|-------------|-----------|
| ITM-2041  | Steel Beam        | A36 Steel          | 30  | Each | 2025-06-10     | Zenith Engineering    | $120.00        | Structural  | ✅         |
| ITM-1932  | Cable Reel        | High Voltage 480V  | 50  | Feet | 2025-06-12     | Pinnacle Systems      | $2.80          | Electrical  | ❌         |
| ITM-1740  | PVC Pipe          | Schedule 40        | 100 | Feet | 2025-06-20     | Vertex Constructors   | $1.45          | Piping      | ❌         |

---

### ✅ Key Output Fields Explained:

- **Suggested Supplier**: Inferred from historical BOMs using string-matching logic.
- **Past Unit Cost**: Serves as a price benchmark during RFQ evaluation.
- **Category**: Tags each item (Mechanical, Electrical, etc.) to allow grouped sourcing.
- **Risk Flag**: Flags missing, conflicting, or ambiguous item data needing review.



# PART 2 - RFQ Preparation

## 📥 Expected Data Inputs for RFQ Preparation (Part 2)

To simulate the RFQ Preparation process, we use structured inputs derived from the final output of Part 1 (Scope Identification). These inputs feed into the creation of RFQ documents, supplier communication templates, and tracking dashboards.

---

### ✅ 1. Scope Document (Main Input)

This is the primary file generated at the end of Part 1 and serves as the base for all RFQ content.

| Column Name             | Description                                                |
|-------------------------|------------------------------------------------------------|
| `Item Code`             | Unique identifier for each part or component               |
| `Item Description`      | Human-readable description of the item                     |
| `Specification`         | Technical or quality-related specification                 |
| `Qty`                   | Quantity to be quoted                                      |
| `UOM` (Unit of Measure) | Unit type (e.g., Each, Feet, Gallon)                       |
| `Delivery Date`         | Expected delivery or fulfillment date                      |
| `Suggested Supplier`    | Recommended supplier based on historical BOM similarity     |
| `Past Unit Cost`        | Used as a price benchmark during quotation evaluation       |
| `Category`              | For grouping items (e.g., Electrical, Mechanical)           |
| `Risk Flag`             | Flags incomplete or ambiguous items needing manual review   |

---

### ✅ 2. RFQ Metadata (Simulated Contextual Inputs)

These fields are simulated at runtime during RFQ generation to define the header and contextual information included in each RFQ document.

| Field Name           | Example Value             | Purpose                                       |
|----------------------|---------------------------|-----------------------------------------------|
| `RFQ ID`             | `RFQ-1001`                | Unique identifier to track the RFQ            |
| `Project Name`       | `Metro Line 9`            | Project or initiative the procurement supports|
| `Procurement Contact`| `procurement@abc.com`     | Point of contact for supplier responses       |
| `Response Deadline`  | `2025-05-15`              | Last date to receive quotes from suppliers    |
| `Payment Terms`      | `Net 30`                  | Payment expectations post-delivery            |
| `Shipping Terms`     | `FOB - Factory`           | Responsibility for logistics and risk         |

---

## ✅ 3. Supplier Master List

This list serves as a centralized directory of suppliers and is used to:

- Match suppliers to relevant scope items (based on expertise or category)
- Customize RFQ emails based on region, certifications, or history
- Simulate supplier filtering or scoring in future stages (like bid evaluation)

### 📄 Column Descriptions

| Column Name        | Description                                                                 |
|--------------------|-----------------------------------------------------------------------------|
| `Vendor Name`      | Official name of the supplier company                                        |
| `Region`           | Supplier’s operating region or delivery base (used for tailoring terms)     |
| `Contact Name`     | Point of contact at the supplier company                                     |
| `Contact Email`    | Email address for sending RFQs and receiving quotations                      |
| `History`          | Brief vendor engagement history (preferred status, delays, onboarding notes)|
| `Certifications`   | Compliance credentials (e.g., ISO, CE, FM) to match project or RFQ needs     |
| `Expertise`        | Areas of supply or technical specialization (e.g., HVAC ducts, Control Cable)|
| `Avg Lead Time (days)` | Average time in days the supplier takes to deliver ordered items           |



---



## TASK 1 - SUPPLIER/VENDOR MASTER LIST

## ✅ Supplier Master List Generation (with GenAI Context)

As part of our procurement automation pipeline, we simulate a **Supplier Master List** that contains detailed supplier profiles, including contact info, expertise, certifications, and AI-generated history. This enriched dataset is used for tailoring RFQ communication, filtering suppliers, and scoring bids in later stages.

---

### 🛠️ How It Works

1. **Sample Pools Defined**
   - `Regions`: USA – East, Brazil, Singapore, etc.
   - `Certifications`: ISO 9001, CE, FM Approved, etc.
   - `Expertise`: Cable Tray, Valves, HVAC Ducts, etc.

2. **Supplier Records Generated**
   - Vendor names and contact names are created using region-aware logic
   - Email addresses are programmatically constructed
   - Certifications and expertise are randomly assigned
   - Average lead time is randomized (12–44 days)

3. **GenAI-Style Vendor History**
   - For each supplier, a 1–2 sentence summary is generated using a mock `get_response()` function.
   - This simulates how a GenAI model would describe the supplier’s reliability, onboarding, or expertise.
   - Example:
     > "Alpha Cable Solutions Ltd., based in USA – East, has a proven track record delivering control cable with certifications such as ISO 9001 and more. Their average lead time is 20 days."

4. **Final Output Fields**

| Column Name           | Description                                                             |
|------------------------|-------------------------------------------------------------------------|
| `Vendor Name`          | Official supplier name                                                  |
| `Region`               | Operating or delivery region                                            |
| `Contact Name`         | Supplier POC (simulated)                                                |
| `Contact Email`        | Email address for RFQ correspondence                                    |
| `Certifications`       | Compliance credentials (e.g., ISO, FM Approved)                         |
| `Expertise`            | Categories of parts or services supplied                                |
| `Avg Lead Time (days)` | Average estimated delivery timeline                                     |
| `History`              | GenAI-style narrative about vendor reliability and past performance     |

---

### 📂 Output File

- `supplier_master_list.csv` — Stored in the `rfq_output/` folder

---

### 📈 Next Usage

This supplier list will be used in:
- Tailoring RFQ documents and emails by category/region
- Filtering suppliers during quote comparison
- Generating a supplier recommendation or scorecard



In [1]:
import numpy as np
import pandas as pd

# Simulated LLM response (replace this with actual get_response() if LLM available)
def get_response(inputs, template, role=None, llm="llama", md=False):
    # MOCK SIMULATION: This should be replaced with actual LLM output
    region, expertise = inputs
    return f"{region} {expertise.split(',')[0]} Solutions Ltd."

def get_name_response(region, template, role=None, llm="llama", md=False):
    # MOCK SIMULATION: This should be replaced with actual LLM output
    fake_names = {
        "Japan": "Hiroshi Tanaka", "India – South": "Arjun Mehta", "Spain": "Rafael Dominguez",
        "USA – East": "Emily Carter", "China": "Mei Lin", "Brazil": "Carlos Silva",
        "Kuwait": "Ali Faris", "Canada": "Sophie Dubois", "Italy": "Giovanni Russo",
        "Mexico": "Luis Herrera", "Singapore": "Jia Wei", "Vietnam": "Thien Bao",
        "Malaysia": "Aiman Razak", "South Korea": "Min-Joon Park", "Saudi Arabia": "Fatimah Al-Dossary",
        "UAE – West": "Zayd Al-Maktoum"
    }
    return fake_names.get(region, f"Contact from {region}")

# Set seed for reproducibility
np.random.seed(42)

# Number of vendor records to simulate
n_vendors = 25

# Sample pools
vendor_regions = [
    "USA – East", "Brazil", "Singapore", "South Korea", "India – South", "Malaysia",
    "Spain", "Saudi Arabia", "Italy", "Kuwait", "Canada", "Japan", "Mexico", "UAE – West", "China", "Vietnam"
]

certification_pool = [
    "ISO 9001", "FM Approved", "UL Listed", "API Monogram",
    "CE", "ASME", "Local Safety Compliance", "Material Test Report"
]

expertise_pool = [
    "Control Cable", "Lighting Fixture", "Structural Steel", "Cable Tray",
    "HVAC Duct Connection", "Flanges", "Valves", "Fire Rated Door", "Earthing Strip"
]

# Initialize empty list to collect each row
vendor_rows = []

for i in range(n_vendors):
    region = np.random.choice(vendor_regions)
    expertise = ", ".join(np.random.choice(expertise_pool, size=np.random.randint(1, 3), replace=False))
    vendor_name = get_response([region, expertise], template="generate realistic vendor name")
    contact_name = get_name_response(region, template="generate realistic contact name")
    certifications = ", ".join(np.random.choice(certification_pool, size=np.random.randint(2, 5), replace=False))
    lead_time = np.random.randint(12, 45)

    vendor_rows.append({
        "Vendor Name": vendor_name,
        "Region": region,
        "Contact Name": contact_name,
        "Contact Email": f"{contact_name.lower().replace(' ', '.')}_{i+1}@{vendor_name.lower().replace(' ', '').replace(',', '')}.com",
        "Certifications": certifications,
        "Expertise": expertise,
        "Avg Lead Time (days)": lead_time
    })

# Create DataFrame
df_vendors = pd.DataFrame(vendor_rows)

# Preview
df_vendors.head()

Vendor Name         Region  \
0                  Spain Flanges Solutions Ltd.          Spain   
1                Vietnam Flanges Solutions Ltd.        Vietnam   
2  India – South Fire Rated Door Solutions Ltd.  India – South   
3     Japan HVAC Duct Connection Solutions Ltd.          Japan   
4    Kuwait HVAC Duct Connection Solutions Ltd.         Kuwait   

       Contact Name                                      Contact Email  \
0  Rafael Dominguez   rafael.dominguez_1@spainflangessolutionsltd..com   
1         Thien Bao        thien.bao_2@vietnamflangessolutionsltd..com   
2       Arjun Mehta  arjun.mehta_3@india–southfirerateddoorsolution...   
3    Hiroshi Tanaka  hiroshi.tanaka_4@japanhvacductconnectionsoluti...   
4         Ali Faris  ali.faris_5@kuwaithvacductconnectionsolutionsl...   

                             Certifications  \
0         ISO 9001, Local Safety Compliance   
1  ISO 9001, ASME, Material Test Report, CE   
2     Material Test Report, ASME, UL Listed   
3                   UL Listed, ISO 9001, CE   
4                       UL Listed, ISO 9001   

                                Expertise  Avg Lead Time (days)  
0                  Flanges, Control Cable                    13  
1               Flanges, Structural Steel                    14  
2                         Fire Rated Door                    18  
3                    HVAC Duct Connection                    29  
4  HVAC Duct Connection, Structural Steel                    40

In [2]:
# Mock get_response to simulate LLM output (replace with actual LLM if available)
def get_response(inputs, template, role=None, llm="llama", md=False):
    vendor_name, region, certifications, expertise, lead_time = inputs
    return f"{vendor_name}, based in {region}, has a proven track record delivering {expertise.lower()} with certifications such as {certifications.split(',')[0]} and more. Their average lead time is {lead_time} days."

# Define vendor history template (optional if using inline like above)
def vendor_history_template(inputs):
    vendor_name, region, certifications, expertise, lead_time = inputs
    return f"""Generate a 1–2 sentence vendor history summary for procurement records.
Vendor Name: {vendor_name}
Region: {region}
Certifications: {certifications}
Expertise: {expertise}
Average Lead Time: {lead_time} days
Make the history sound realistic and professional — referencing onboarding, reliability, or performance if possible."""

# Apply LLM-generated history to the dataframe
df_vendors["History"] = df_vendors.apply(
    lambda row: get_response(
        [
            row["Vendor Name"],
            row["Region"],
            row["Certifications"],
            row["Expertise"],
            row["Avg Lead Time (days)"]
        ],
        template=vendor_history_template,
        role="You are a helpful procurement assistant generating brief vendor history notes.",
        llm="llama",
        md=False,
    ),
    axis=1
)

# Preview the updated DataFrame
df_vendors[["Vendor Name", "Region","Contact Name","Contact Email", "Expertise","Certifications", "Avg Lead Time (days)", "History"]].head(20)


Vendor Name         Region  \
0                   Spain Flanges Solutions Ltd.          Spain   
1                 Vietnam Flanges Solutions Ltd.        Vietnam   
2   India – South Fire Rated Door Solutions Ltd.  India – South   
3      Japan HVAC Duct Connection Solutions Ltd.          Japan   
4     Kuwait HVAC Duct Connection Solutions Ltd.         Kuwait   
5          China Lighting Fixture Solutions Ltd.          China   
6            Spain Earthing Strip Solutions Ltd.          Spain   
7      Spain HVAC Duct Connection Solutions Ltd.          Spain   
8             Spain Control Cable Solutions Ltd.          Spain   
9        Vietnam Lighting Fixture Solutions Ltd.        Vietnam   
10                 Canada Flanges Solutions Ltd.         Canada   
11      UAE – West Earthing Strip Solutions Ltd.     UAE – West   
12       UAE – West Control Cable Solutions Ltd.     UAE – West   
13         Canada Fire Rated Door Solutions Ltd.         Canada   
14           Saudi Arabia Flanges Solutions Ltd.   Saudi Arabia   
15     USA – East Fire Rated Door Solutions Ltd.     USA – East   
16              UAE – West Valves Solutions Ltd.     UAE – West   
17             UAE – West Flanges Solutions Ltd.     UAE – West   
18        Canada Structural Steel Solutions Ltd.         Canada   
19              Singapore Flanges Solutions Ltd.      Singapore   

          Contact Name                                      Contact Email  \
0     Rafael Dominguez   rafael.dominguez_1@spainflangessolutionsltd..com   
1            Thien Bao        thien.bao_2@vietnamflangessolutionsltd..com   
2          Arjun Mehta  arjun.mehta_3@india–southfirerateddoorsolution...   
3       Hiroshi Tanaka  hiroshi.tanaka_4@japanhvacductconnectionsoluti...   
4            Ali Faris  ali.faris_5@kuwaithvacductconnectionsolutionsl...   
5              Mei Lin    mei.lin_6@chinalightingfixturesolutionsltd..com   
6     Rafael Dominguez  rafael.dominguez_7@spainearthingstripsolutions...   
7     Rafael Dominguez  rafael.dominguez_8@spainhvacductconnectionsolu...   
8     Rafael Dominguez  rafael.dominguez_9@spaincontrolcablesolutionsl...   
9            Thien Bao  thien.bao_10@vietnamlightingfixturesolutionslt...   
10       Sophie Dubois    sophie.dubois_11@canadaflangessolutionsltd..com   
11     Zayd Al-Maktoum  zayd.al-maktoum_12@uae–westearthingstripsoluti...   
12     Zayd Al-Maktoum  zayd.al-maktoum_13@uae–westcontrolcablesolutio...   
13       Sophie Dubois  sophie.dubois_14@canadafirerateddoorsolutionsl...   
14  Fatimah Al-Dossary  fatimah.al-dossary_15@saudiarabiaflangessoluti...   
15        Emily Carter  emily.carter_16@usa–eastfirerateddoorsolutions...   
16     Zayd Al-Maktoum  zayd.al-maktoum_17@uae–westvalvessolutionsltd....   
17     Zayd Al-Maktoum  zayd.al-maktoum_18@uae–westflangessolutionsltd...   
18       Sophie Dubois  sophie.dubois_19@canadastructuralsteelsolution...   
19             Jia Wei       jia.wei_20@singaporeflangessolutionsltd..com   

                                 Expertise  \
0                   Flanges, Control Cable   
1                Flanges, Structural Steel   
2                          Fire Rated Door   
3                     HVAC Duct Connection   
4   HVAC Duct Connection, Structural Steel   
5                         Lighting Fixture   
6                           Earthing Strip   
7                     HVAC Duct Connection   
8                            Control Cable   
9       Lighting Fixture, Structural Steel   
10                                 Flanges   
11                          Earthing Strip   
12                   Control Cable, Valves   
13                         Fire Rated Door   
14                Flanges, Fire Rated Door   
15       Fire Rated Door, Lighting Fixture   
16                                  Valves   
17                                 Flanges   
18      Structural Steel, Lighting Fixture   
19               Flanges, Lighting Fixture   

                                       Certi

## TASK 2 - RFQ Metadata

## 📄 Sample RFQ Metadata

This RFQ metadata outlines the contextual and commercial details that will be embedded in the RFQ document sent to suppliers such as:

- Spain Flanges Solutions Ltd.
- Vietnam Flanges Solutions Ltd.
- India – South Fire Rated Door Solutions Ltd.
- Japan HVAC Duct Connection Solutions Ltd.
- Kuwait HVAC Duct Connection Solutions Ltd.

---

### 🧾 RFQ Metadata for RFQ-1001

| Field                  | Value                                      |
|------------------------|--------------------------------------------|
| **RFQ ID**             | RFQ-1001                                   |
| **Project Name**       | Metro Line 9                               |
| **Generated On**       | 2025-05-08                                  |
| **Procurement Contact**| arjun.mehta@abcinfra.com                   |
| **Response Deadline**  | 2025-05-15                                  |
| **Payment Terms**      | Net 30                                     |
| **Shipping Terms**     | FOB – Factory                              |
| **Delivery Window**    | Between 2025-06-01 and 2025-06-30           |
| **Evaluation Criteria**| Price, Certifications, Lead Time, Expertise|

---

### 📝 Instructions for Suppliers

Please review the attached RFQ and submit your quotation by **2025-05-15**. Include:

- Unit price and total for each item
- Lead time (in calendar days)
- Validity of quote
- Any exceptions or clarifications

All quotes must be sent to: **arjun.mehta@abcinfra.com**



In [3]:
import pandas as pd
from datetime import datetime, timedelta
import os

# Set folder path
output_path = "rfq_output"
os.makedirs(output_path, exist_ok=True)

# Today's date and response deadline
generated_on = datetime.today().strftime('%Y-%m-%d')
response_deadline = (datetime.today() + timedelta(days=7)).strftime('%Y-%m-%d')

# Simulated RFQ Metadata
rfq_metadata = {
    "RFQ ID": "RFQ-1001",
    "Project Name": "Metro Line 9",
    "Generated On": generated_on,
    "Procurement Contact": "arjun.mehta@abcinfra.com",
    "Response Deadline": response_deadline,
    "Payment Terms": "Net 30",
    "Shipping Terms": "FOB – Factory",
    "Delivery Window": "Between 2025-06-01 and 2025-06-30",
    "Evaluation Criteria": "Price, Certifications, Lead Time, Expertise"
}

# Convert to DataFrame
rfq_meta_df = pd.DataFrame([rfq_metadata])

rfq_meta_df


RFQ ID  Project Name Generated On       Procurement Contact  \
0  RFQ-1001  Metro Line 9   2025-05-08  arjun.mehta@abcinfra.com   

  Response Deadline Payment Terms Shipping Terms  \
0        2025-05-15        Net 30  FOB – Factory   

                     Delivery Window  \
0  Between 2025-06-01 and 2025-06-30   

                           Evaluation Criteria  
0  Price, Certifications, Lead Time, Expertise

## ✅ Conclusion: Completion of Data Simulation for Scope Identification and RFQ Preparation

In this stage, we focused exclusively on simulating the structured datasets required to support an end-to-end GenAI-powered procurement pipeline.

### 🎯 Objective:
To generate clean, realistic, and modular data artifacts that mimic what a real procurement system would consume in the **Scope Identification** and **RFQ Preparation** stages.

---

### 🔹 What We Simulated

#### 🧩 **Part 1: Scope Identification**
- Engineering BOMs (`.xlsx` and `.pdf`) with items, specs, and delivery dates
- Auto-generated `.dxf` drawings to simulate CAD parsing
- Historical BOMs to provide supplier/cost recommendations
- A final **Scope Document** enriched with:
  - Suggested suppliers
  - Past unit costs
  - Risk flags
  - Procurement categories

#### 📄 **Part 2: RFQ Preparation**
- **RFQ Metadata** including RFQ ID, project name, deadline, payment/shipping terms
- A **Supplier Master List** featuring:
  - Region, contact info, certifications, lead time
  - GenAI-style history for supplier profiling

---

### 🗂️ Final Simulated Outputs

| Artifact                        | Purpose                                      |
|--------------------------------|----------------------------------------------|
| `BOM_*.xlsx/.pdf`              | Raw item requirements from engineering       |
| `generated_dxf/`               | Simulated drawings for parsing               |
| `Old_BOM_*.xlsx/.pdf`          | Historical reference for supplier matching   |
| `Scope_Document_BOM_1.xlsx`    | Cleaned, enriched procurement scope          |
| `rfq_metadata_RFQ1001.csv`     | Project-level terms & context for RFQ        |
| `supplier_master_list.csv`     | Supplier profiles with GenAI-generated notes |

---

### Next Steps (Not Yet Implemented):
- Generate vendor-facing **RFQ PDF/DOCX**
- Simulate **email drafts or RFQ distribution**
- Create quote tracking & **bid evaluation system**

This simulation sets the foundation for automating intelligent procurement flows driven by GenAI.

